# Homework 2 Part 2

In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import KFold
%matplotlib inline

#### Q1 (2 points)

a. Implement a function that takes in the *train.csv* dataset and returns a cleaned dataset without outliers and NAs.

In [262]:
train = pd.read_csv('train.csv')

def preprocessing_pipeline(train):
    ## Your code goes here
    train = train.dropna() #removes all NAs
    train = train.drop(columns=['Id'], axis=1) #remove ID column, unnecessay 
    
    #remove outliers
    for column in train:
        
        # Calculate the mean and standard deviation
        mu = np.mean(train[column])
        std = np.std(train[column])

        # Normalize the data
        normalized_data = (train[column] - mu)/std

        # Find the data index that is 3 standard deviation away from the distribution
        indexes = normalized_data < 3
        train = train.loc[indexes, :]
        processed_train = train
    
    return processed_train
train_clean = preprocessing_pipeline(train)
train_clean.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,12.0,0.37,0.76,4.2,0.066,7.0,38.0,1.00040,3.22,0.60,13.0,7
1,7.1,0.69,0.08,2.1,0.063,42.0,52.0,0.99608,3.42,0.60,10.2,6
2,9.6,0.50,0.36,2.8,0.116,26.0,55.0,0.99722,3.18,0.68,10.9,5
3,7.7,0.96,0.20,2.0,0.047,15.0,60.0,0.99550,3.36,0.44,10.9,5
5,11.9,0.38,0.49,2.7,0.098,12.0,42.0,1.00040,3.16,0.61,10.3,5


b. Implement a function that takes in the R<sup>2</sup> of a regression and return the VIF value. 

In [263]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1-r_squared)
    
    return vif


c. Implement a function that takes in preprocessed data and returns a dataframe that contains the VIF for each variable. The dataframe should contain two columns: variable names and their VIFs.

In [264]:
def generate_vif_dataframe(processed_train):
    ## Your code goes here
    newData = []
    columnArray = []

    vif_train = processed_train.drop(['quality'], axis = 1)
    columns = vif_train.columns
    
    for col in columns:
        
        columnArray = [col]
        X = vif_train.drop(col,axis=1).values
        y = vif_train[col].values
        
        #calculate regression
        heights_constant_added = sm.add_constant(X)
        results = sm.OLS(y, heights_constant_added).fit()
        r2 = results.rsquared
        
        vif = round(calculate_vif(r2), 3)
        columnArray.append(vif)
        newData.append(columnArray)
        
        vif_dataframe = pd.DataFrame(newData, columns = ['Variable', 'VIF'])   
    
    
    return vif_dataframe



d. Use the functions that you developed from Q1 and generate a VIF dataframe for all the variables in the wine dataset. Discuss the VIF values you find and the effect of multicollinearity on regression models.

In [265]:
## Your code goes here
generate_vif_dataframe(train_clean)


,Variable,VIF
0,fixed acidity,7.884
1,volatile acidity,1.875
2,citric acid,3.209
3,residual sugar,1.651
4,chlorides,1.183
5,free sulfur dioxide,1.976
6,total sulfur dioxide,2.204
7,density,6.251
8,pH,3.231
9,sulphates,1.253


In [195]:
## Your discussion goes here

#There are 2 variables where their VIF is > 5, this means that multicollinearity is present in my model. Fixed Acidity has a VIF
#of 7.884 and Density has a VIF of 6.251. Multicollinearity means that 2 or more predictor variables are highly correlated.
#This is typically bad because it effects our judgement as to which variables are important to our overall model. 
#Multicollinearity also means that there is weak support for our regression plane.

#### Q2 (4 points)

a. Use the VIF dataframe you generated from Q1 and choose a subset of variables that does not have high multicollinearity.

In [266]:
## Your code goes here

#subset of variables that do not have high multicollinearity i.e. VIF < 5 (took out fixed acidity and density)

X = train_clean[["volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide","pH", "sulphates","alcohol"]]


b. Fit a multiple linear regression model and print a summary of the regression result.

In [267]:
## Your code goes here
y=train_clean["quality"]

results = sm.OLS(y, sm.add_constant(X)).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                quality   R-squared:                       0.385
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     68.45
Date:                Sun, 16 Feb 2020   Prob (F-statistic):           1.03e-97
Time:                        23:41:38   Log-Likelihood:                -945.07
No. Observations:                 995   AIC:                             1910.
Df Residuals:                     985   BIC:                             1959.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    4.2764      0.613      6.980      0.000       3.074       5.479
volatile acidity        -1.0243      0.153     -6.682      0.000      -1.325      -0.723
citric acid             -0.3771      0.157     -2.397      0.017      -0.686      -0.068
residual sugar           0.0088      0.024      0.363      0.717      -0.039       0.056
chlorides               -2.5190      0.999     -2.521      0.012      -4.479      -0.559
free sulfur dioxide      0.0040      0.003      1.318      0.188      -0.002       0.010
total sulfur dioxide    -0.0026      0.001     -2.536      0.011      -0.005      -0.001
pH                      -0.6287      0.173     -3.642      0.000      -0.967      -0.290
sulphates                1.4980      0.173      8.683      0.000       1.159       1.837
alcohol                  0.3245      0.023     14.287      0.000       0.280       0.369
==============================================================================
Omnibus:                        6.378   Durbin-Watson:                   2.073
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                8.372
Skew:                          -0.003   Prob(JB):                       0.0152
Kurtosis:                       3.449   Cond. No.                     2.77e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.77e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

c. Use 5-fold cross validation to validate the mean squared error (MSE) of a multiple linear regression model fit to the subset of data. (HINT: You will need to implement a function to calculate the MSE. See this link for more details: https://en.wikipedia.org/wiki/Mean_squared_error).

In [268]:
## Your code goes here
# kfold function will split the data "x" into "k" equal-sized data sets
def kfold(X,k):
    i = 0 
    while i < k:
        xtrain = np.split(X,k)
        return xtrain
        i += 1

# linreg function performs a multi-linear regression thru the x data sets & return the coefficients
def linreg(X,y):
    s = sm.add_constant(X)
    r = sm.OLS(y,s).fit()
    return r.params

# Calculating yhat = constant + b1*x1 + b2*x2 + b3*x3 + b4*x4 + b5*x5 +b6*x6 +b7*x7 + b8*x8 + b9*x9
def yhat(parameters,X):
    yhat = parameters[0] + parameters[1]*X.iloc[:,0] + parameters[2]*X.iloc[:,1] + parameters[3]*X.iloc[:,2] + parameters[4]*X.iloc[:,3] + parameters[5]*X.iloc[:,4] + parameters[6]*X.iloc[:,5] + parameters[7]*X.iloc[:,6] + parameters[8]*X.iloc[:,7]+parameters[9]*X.iloc[:,8]
    return yhat

#Calculating MSE 
def mse(yi,yhat):
    n = len(yi)
    mse_score = (1/n) * np.sum((yi - yhat)**2)
    return mse_score
      
t1,t2,t3,t4,t5 = kfold(X,5)
q1,q2,q3,q4,q5 = kfold(y,5)


# Implementing 5-fold cross validation

#Round 1: 5th fold = test set
xtrain = [t1,t2,t3,t4]
ytrain = [q1,q2,q3,q4]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = linreg(xtrain_set,ytrain_set)

yhat_train = yhat(p,t5)
mse1 = mse(q5,yhat_train)

#Round 2: 4th fold = test set
xtrain = [t1,t2,t3,t5]
ytrain = [q1,q2,q3,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = linreg(xtrain_set,ytrain_set)

yhat_train = yhat(p,t4)
mse2 = mse(q4,yhat_train)

#Round 3: 3rd fold = test set
xtrain = [t1,t2,t4,t5]
ytrain = [q1,q2,q4,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = linreg(xtrain_set,ytrain_set)

yhat_train = yhat(p,t3)
mse3 = mse(q3,yhat_train)

#Round 4: 2nd fold = test set
xtrain = [t1,t3,t4,t5]
ytrain = [q1,q3,q4,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = linreg(xtrain_set,ytrain_set)

yhat_train = yhat(p,t2)
mse4 = mse(q2,yhat_train)

#Round 5: 1st fold = test set
xtrain = [t2,t3,t4,t5]
ytrain = [q2,q3,q4,q5]
xtrain_set = pd.concat(xtrain) 
ytrain_set = pd.concat(ytrain)
p = linreg(xtrain_set,ytrain_set)

yhat_train = yhat(p,t1)
mse5 = mse(q1,yhat_train)

average_mse = (mse1 + mse2 + mse3 + mse4 + mse5)/5



# Calculating the MSE between the actual quality given & the estimated quality from multi-linear regression
print("The average MSE is:",average_mse)

The average MSE is: 0.4021195039657126


d. Make a prediction on the test set using the model you fit in 2b and submit to Kaggle.

In [269]:
## Your code goes here

test = pd.read_csv('test.csv')

x2test = test[["volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide","pH", "sulphates","alcohol"]]

x2train = train_clean[["volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide","pH", "sulphates","alcohol"]]
quality = train_clean['quality']

new_para = linreg(x2train,quality)
pred = yhat(new_para,x2test)

test['predicted'] = pred

#submit to kaggle
sample_submission = pd.read_csv('sample_submission.csv')

# Create a dummy submission that has entries as many as the test set.
y_pred = np.random.rand(test.shape[0]) * 10 # Create random numbers from 0-10 as dummy solution
sample_submission.loc[:, 'Predicted'] = pred # Change the Predicted column to your prediction
sample_submission.head()
sample_submission.to_csv('Goldie_Malamud_HW2(1).csv', header=True, index=False)



e. Compare the Kaggle score and the cross-validation result. Is the cross-validation result a good representation of the prediction accuracy on the test data?

In [ ]:
## Your discussion here
'''
Kaggle Score = 0.47871
cross-validation MSE = 0.40211

Yes, I believe that the cross-validation result is a good representation of the prediction accuaracy on the test data. 
As you can see, the kaggle score and the MSE and only 0.07 apart.
'''

f. Discuss the difference between goodness of fit on the training set, on the validation set, and on the test set.

In [ ]:
## Your discussion here
'''
Goodness of fit on the training set means how "well"/how much of the variance of the training set is captured in the model created.
Whereas goodness of fit on the validation set refers to how well your model has been trained. And finally, goodness of fit
on the test set refers to how well the model can be applied to "future data" or "real-world data" and comparing with the outcomes
when they occur. 
'''